In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn import metrics

import sys, os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import math
import pickle

In [ ]:
#https://github.com/dhaitz/mplcyberpunk
import mplcyberpunk

In [ ]:
sys.path.append(os.path.realpath('..'))

In [ ]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
xgb_train_df = pd.read_csv(r'rendered_data/xgbregressor_train_submission.csv', index_col=0)
lgbm_train_df = pd.read_csv(r'rendered_data/lgbmregressor_train_submission.csv', index_col=0)
cat_train_df = pd.read_csv(r'rendered_data/catregressor_train_submission.csv', index_col=0)
lassocv_train_df = pd.read_csv(r'rendered_data/lassocvregressor_train_submission.csv', index_col=0)

xgb_test_df = pd.read_csv(r'rendered_data/xgbregressor_test_submission.csv', index_col=0)
lgbm_test_df = pd.read_csv(r'rendered_data/lgbmregressor_test_submission.csv', index_col=0)
cat_test_df = pd.read_csv(r'rendered_data/catregressor_test_submission.csv', index_col=0)
lassocv_test_df = pd.read_csv(r'rendered_data/lassocvregressor_test_submission.csv', index_col=0)

In [ ]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [ ]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [ ]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

In [ ]:
def roc_auc_score(y_valid, preds):
    fpr, tpr, _ = roc_curve(y_valid, preds)
    roc_auc = auc(fpr, tpr)
    return roc_auc, fpr, tpr

In [ ]:
with open(f"../../BlendingSets/feature_{4}_depth_{1}.hex", "rb") as input_file:
    combos = pickle.load(input_file)
len(combos)

In [ ]:
#combos = np.array(list(combos))*0.01

In [ ]:
combos

In [ ]:
coefficients = combos
best_score = 0
best_coefficients_index = 0
plot_data = None

n_cols = 6

n_rows = 1
cell_with_dim = 4
cell_height_dim = 3
plt.style.use("cyberpunk")
fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * cell_with_dim, n_rows * cell_height_dim))

roc_auc, fpr, tpr = roc_auc_score(train_df[target], xgb_train_df[target])
sns.lineplot(y=tpr, x=fpr, label=f'xgb {roc_auc}', ax=ax[0])

mplcyberpunk.add_underglow(ax[0])
mplcyberpunk.make_lines_glow(ax[0])
sns.lineplot(y=tpr, x=fpr, label=f'xgb {roc_auc}', ax=ax[5])

roc_auc, fpr, tpr = roc_auc_score(train_df[target], lgbm_train_df[target])
sns.lineplot(y=tpr, x=fpr, label=f'lgbm {roc_auc}', ax=ax[1])
mplcyberpunk.add_underglow(ax[1])
mplcyberpunk.make_lines_glow(ax[1])
sns.lineplot(y=tpr, x=fpr, label=f'lgbm {roc_auc}', ax=ax[5])

roc_auc, fpr, tpr = roc_auc_score(train_df[target], cat_train_df[target])
sns.lineplot(y=tpr, x=fpr, label=f'cat {roc_auc}', ax=ax[2])
mplcyberpunk.add_underglow(ax[2])
mplcyberpunk.make_lines_glow(ax[2])
sns.lineplot(y=tpr, x=fpr, label=f'cat {roc_auc}', ax=ax[5])

roc_auc, fpr, tpr = roc_auc_score(train_df[target], lassocv_train_df[target])
sns.lineplot(y=tpr, x=fpr, label=f'lassocv {roc_auc}', ax=ax[3])
mplcyberpunk.add_underglow(ax[3])
mplcyberpunk.make_lines_glow(ax[3])
sns.lineplot(y=tpr, x=fpr, label=f'lassocv {roc_auc}', ax=ax[5])


ax[2].set_title('AUC')
ax[2].set_xlabel('FP')
ax[0].set_ylabel('TP')

for index, (a, b, c, d) in tqdm(enumerate(coefficients)):
    roc_auc, fpr, tpr = roc_auc_score(train_df[target], (a * xgb_train_df[target]) + (b * lgbm_train_df[target]) + (c * cat_train_df[target]) + (d * lassocv_train_df[target]))
    if roc_auc > best_score:
        best_score = roc_auc
        best_coefficients_index = index
        plot_data = [roc_auc, fpr, tpr]

sns.lineplot(y=plot_data[2], x=plot_data[1], label=f'blend {plot_data[0]}', ax=ax[4])
mplcyberpunk.add_underglow(ax[4])
mplcyberpunk.make_lines_glow(ax[4])
sns.lineplot(y=plot_data[2], x=plot_data[1], label=f'blend {plot_data[0]}', ax=ax[5])

plt.show()
print(f"Best coef:{combos[best_coefficients_index]} w/ score:{best_score}")

In [ ]:
coef = combos[best_coefficients_index]
#blended_test_results = xgb_test_df[target]*.25 + lgbm_test_df[target]*.25 + cat_test_df[target]*.25+ lassocv_test_df[target]*.25
#blended_test_results = xgb_test_df[target]*0 + lgbm_test_df[target]*0 + cat_test_df[target]*0+ lassocv_test_df[target]*1
blended_test_results = xgb_test_df[target]*coef[0] + lgbm_test_df[target]*coef[1] + cat_test_df[target]*coef[2] + lassocv_test_df[target]*coef[3]
final_blended_test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: blended_test_results[target[0]].values})
final_blended_test_submission.to_csv(fr'rendered_data/final_blended_test_submission.csv', index=False)

In [ ]:
from itertools import islice, cycle

plt.style.use("cyberpunk")

def add_secondary_plot(df, column, target_column, ax, n_bins, color=3, show_yticks=False, marker="."):
    secondary_ax = ax.twinx()
    bins = pd.cut(df[column], bins=n_bins)
    bins = pd.IntervalIndex(bins)
    bins = (bins.left + bins.right) / 2
    target = df.groupby(bins)[target_column].mean()
    target.plot(
        ax=secondary_ax, linestyle='',
        marker=marker, color=color, label=f"Mean '{target_column}'"
    )
    secondary_ax.grid(visible=False)
    
    if not show_yticks:
        secondary_ax.get_yaxis().set_ticks([])
        
    return secondary_ax

def render_feature_distros(train_df, test_df, features=[], labels=[], n_bins=50, n_cols=4, pad=2, h_pad=4, w_pad=None):
    histplot_hyperparams = {
        'kde':True,
        'alpha':0.4,
        'stat':'percent',
        'bins':n_bins
    }
    markers = ['.', '+', 'x', '1', '2']
    
    n_rows = math.ceil(len(features) / n_cols)
    cell_with_dim = 4
    cell_height_dim = 3
    
    fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * cell_with_dim, n_rows * cell_height_dim))
    plt.tight_layout(pad=pad, h_pad=h_pad, w_pad=w_pad, rect=None)
    
    # delete exess subplots
    for a in ax[n_rows - 1, int(((n_rows - (len(features) / n_cols)) * n_cols*-1)):]:
        a.axis('off')
        
    leg_handles = []
    leg_labels = []
    
    axs = []

    for i, feature in enumerate(features):
        row = math.ceil(i / n_cols) - 1
        col = (i % n_cols)
        
        color_cycle = islice(mplcyberpunk.cyberpunk_stylesheets['cyberpunk']['axes.prop_cycle'], 0, None)
        
        sns.histplot(train_df[feature], label='Train X', ax=ax[row, col], color=next(color_cycle)['color'], **histplot_hyperparams)
        sns.histplot(test_df[feature], label='Test X', ax=ax[row, col],color=next(color_cycle)['color'], **histplot_hyperparams)
        ax[row, col].set_title(f'{feature} Distribution')
        mplcyberpunk.make_lines_glow(ax[row, col])
        axs.append(ax[row, col].get_legend_handles_labels())

        for j, label in enumerate(labels):
            sub_ax = add_secondary_plot(train_df, feature, label, ax[row, col], n_bins, color=next(color_cycle)['color'], marker=markers[j])
            axs.append(sub_ax.get_legend_handles_labels())
        
    for axis in axs:
        if axis[1][0] not in leg_labels:
            leg_labels.extend(axis[1])
            leg_handles.extend(axis[0])
        
    fig.legend(leg_handles, leg_labels, loc='upper center', bbox_to_anchor=(0.5, 1.04), fontsize=14, ncol=len(features) + 2)

In [ ]:
train_df[f'XGB_{target[0]}'] = xgb_train_df[target]
train_df[f'LGBM_{target[0]}'] = lgbm_train_df[target]
train_df[f'CAT_{target[0]}'] = cat_train_df[target]
train_df[f'LASSOCV_{target[0]}'] = lassocv_train_df[target]

In [ ]:
render_feature_distros(train_df=train_df, test_df=test_df, features=features, labels=[target[0], f'XGB_{target[0]}', f'LGBM_{target[0]}', f'CAT_{target[0]}'])